In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the csv files
world_population = pd.read_csv('world_population.csv')
world_bank = pd.read_csv('world_bank.csv')

In [3]:
# Drop the years we don't have data for from world_bank
available_years = [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2022]
world_bank = world_bank[world_bank['Year'].isin(available_years)]

In [4]:
# Drop all countries/territories with a population less than 100,000
world_population = world_population[world_population['2022 Population'] > 100000]

In [5]:
# Rename the country column names for consistency
world_population.rename(columns={'Country/Territory': 'Country'}, inplace=True)
world_bank.rename(columns={'Country Name': 'Country'}, inplace=True)

In [6]:
# Melt the population data so each row is each ear for each country
world_population_years = ['1970 Population', '1980 Population', '1990 Population', '2000 Population', '2010 Population', '2015 Population',
                          '2020 Population', '2022 Population']

world_population_long = world_population.melt(
    id_vars=['Country'],
    value_vars=world_population_years,
    var_name='Year',
    value_name='Population'
)

world_population_long['Year'] = world_population_long['Year'].str.split().str[0].astype(int)

In [7]:
# Merge the data
merged_data = pd.merge(
    world_population_long,
    world_bank,
    on=['Country', 'Year'],
    how='inner'
)

# Sort the data by country, then year
merged_data = merged_data.sort_values(by=['Country', 'Year'])
merged_data = merged_data.reset_index(drop=True)

In [8]:
# Add a column to display the population 10 years later (5 for 2010)
merged_data['Next Population'] = merged_data.groupby('Country')['Population'].shift(-1)

# Add the growth rate
merged_data['Growth Rate'] = (merged_data['Next Population'] - merged_data['Population']) / merged_data['Population']

In [10]:
# Reorder the columns for convenience
cols = list(merged_data.columns)
cols.remove('Next Population')
cols.remove('Growth Rate')
cols.insert(3, 'Next Population')
cols.insert(4, 'Growth Rate')
merged_data = merged_data[cols]

In [12]:
# Export csv file
merged_data.to_csv("merged_data_jacob.csv", index=False)